In [1]:
import pandas as pd
import sys
# if there are no conflicting packages in the default Python Libs =>
sys.path.append("/usr/home/hskimim/pdfminer")
import re
import extract_reference_type1 as ref1
import extract_reference_type2 as ref2
import os

In [2]:
dict_df = pd.read_csv('../make_dictionary_process/well_made_dictionary.csv')
dict_df.tail()

,char,degree,journal,name,paper_name,school,year
28910,[학위논문],석사,NaN,김동현,국내 민간의료보험 시장의 역선택에 관한 실증연구,"경희대학교 대학원,",2008
28911,[학위논문],석사,NaN,이수진,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,"연세대학교 대학원,",2012
28912,[학위논문],박사,NaN,한영하,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,"연세대학교 대학원,",2014
28913,[학위논문],박사,NaN,"Low, A",Essays on asymmetric information in internatio...,"University of California,",1992
28914,[학위논문],박사,NaN,"Averbukh, V",Pricing American options using Monte Carlo sim...,"Cornell University,",1997


In [3]:
file_path = os.listdir('../failure_paper_list/')
file_path = ['../failure_paper_list/' + i for i in file_path]
len(file_path)

458

In [4]:
refer_ls , error_ls = [],[]

for path in file_path : 
    try : 
        file_ = ref1.extract_references_file(path)
        refer_ls.append(ref1.total_parsing_sequence(file_))
    except : error_ls.append(path);print('error!')

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!


#### error_ls에는 인코딩 문제와 Parsing 문제 둘 중 하나가 발생한 폴더들이 들어있다.

In [5]:
error_ls

['../failure_paper_list/20180810121649_JF.txt',
 '../failure_paper_list/vol9_no1_1.txt',
 '../failure_paper_list/vol22_no3_1.txt',
 '../failure_paper_list/±¹¹Î¿¬±Ý_ÀÇ°á±Ç_Çà»ç_°³¼±_¹æ¾È.doc',
 '../failure_paper_list/c89ea226_문규현-홍정효-한덕희.txt',
 '../failure_paper_list/vol14_no2_49.txt',
 '../failure_paper_list/vol11_no1_1.txt',
 '../failure_paper_list/vol18_no2_289.txt',
 '../failure_paper_list/vol13_no2_245.txt',
 '../failure_paper_list/8eaf46b4_유시용.txt',
 '../failure_paper_list/20180725162834_RU.txt',
 '../failure_paper_list/Institutional_Ownership_and_Accounting¡¤¡¤.txt',
 '../failure_paper_list/vol23_no3_213.txt',
 '../failure_paper_list/vol23_no3_249.txt',
 '../failure_paper_list/20180810121649_DH.txt',
 '../failure_paper_list/vol10_no1_37.txt']

In [6]:
len(refer_ls)

442

In [7]:
import numpy as np

### 실제 Raw 데이터를 Parsing 했으니, 딕셔너리에서 뽑기 용이하게 소문자로 변환시켜주고, 양 측의 공백을 제거해준다.

In [9]:
refer_parsing_ls = [j for i in refer_ls for j in i]
len(refer_parsing_ls)

14778

In [11]:
small_refer_ls = [i.strip().lower() for i in refer_parsing_ls]

______________

### 하나의 이름에 경우의 수가 산재한다.

In [37]:
set([i for i in dict_df['name'].tolist() if 'Fama' in i])

{' Fama',
 ' Fama E',
 ' Fama E. F.',
 ' Fama Eugene F',
 ' Fama, E',
 ' Fama, E, F.',
 ' Fama, E.',
 ' Fama, E. F',
 ' Fama, E. F.',
 ' Fama, Eugen F',
 ' Fama, Eugene',
 ' Fama, Eugene F',
 ' Fama, Eugene F.',
 ' Fama, F. Eugene',
 ' Fama,E.',
 ' Fama. E',
 ' Fama. E. F'}

#### 함수 tuning_name은 위와 같은 경우에 Fama 만 Extracting 하는 함수이다.

In [102]:
def tuning_name(name):
    return [i for i in name.replace(',','.').replace(' ','.').split('.') if i][0]

In [109]:
tuning_name(' Fama. E. F') , tuning_name(' Fama. Eugine. F')

('Fama', 'Fama')

In [117]:
dict_df['name'].tolist()[:10]

[' 고봉찬',
 ' 손병철',
 ' Amihud, Yakov',
 ' Ball, Ray, S',
 ' Basu, Sudipta',
 ' Bhardwaj, Ravinder K',
 ' Bhushan, Ravi',
 ' Collins, Daniel W',
 ' Daniel, Kent',
 ' DeBondt, Werner F. M']

In [116]:
[tuning_name(i).strip() if (',' in i) or ('.' in i) else i.strip() for i in dict_df['name'].tolist()][:10]

['고봉찬',
 '손병철',
 'Amihud',
 'Ball',
 'Basu',
 'Bhardwaj',
 'Bhushan',
 'Collins',
 'Daniel',
 'DeBondt']

In [118]:
new_name_ls = [tuning_name(i).strip() if (',' in i) or ('.' in i) else i.strip() for i in dict_df['name'].tolist()]

______________________

### 딕셔너리에서 논문의 이름 , 튜닝된 논문의 저자 , 논문이 이슈된 년도를 묶어서 리스트로 만들어준다.

In [120]:
dict_ls = list(zip(dict_df['paper_name'].tolist(),new_name_ls,dict_df['year'].tolist()))
dict_ls[:5]

[('발생액 이상현상에 대한 위험평가', '고봉찬', 2007),
 ('발생액의 효율적 평가에 영향을 미치는 요인', '손병철', 2005),
 ('Illiquidity and Stock Returns: Cross-Section and Time- Series Effects',
  'Amihud',
  2002),
 ('Problems in Measuring Portfolio Performance: An Application to Contrarian Investment Strategies',
  'Ball',
  1995),
 ('What Do We Learn from Two New Accounting-Based Stock Market Anomalies?',
  'Basu',
  2004)]

### Raw_data 를 소문자 변환시켜주었기 때문에, 딕셔너리 데이터도 모두 소문자 변환시켜준다.

In [129]:
tuned_dict_ls = [(i[0].strip().lower(),i[1].lower(),str(i[2])) for i in dict_ls]
tuned_dict_ls[:5]

[('발생액 이상현상에 대한 위험평가', '고봉찬', '2007'),
 ('발생액의 효율적 평가에 영향을 미치는 요인', '손병철', '2005'),
 ('illiquidity and stock returns: cross-section and time- series effects',
  'amihud',
  '2002'),
 ('problems in measuring portfolio performance: an application to contrarian investment strategies',
  'ball',
  '1995'),
 ('what do we learn from two new accounting-based stock market anomalies?',
  'basu',
  '2004')]

### 실제 딕셔너리에는 Frequency 가 존재한다. 즉, Unique 만 있는 것이 아니다. 이에 따라서 Paper_name 을 기준으로 Unique 하게 만들어준다.

In [134]:
[refer for refer in refer_parsing_ls \
if (tuned_dict_ls[0][0] in refer) and (tuned_dict_ls[0][1] in refer) and (tuned_dict_ls[0][2] in refer)]

['고봉찬 김진우 “발생액 이상현상에 대한 위험평가” 증권학회지 제36권 3호(2007) pp. 425-461',
 '고봉찬 김진우 “발생액 이상현상에 대한 위험평가” 증권학회지 제36권 제3호(2007)pp. 425-461',
 '고봉찬 김진우 “발생액 이상현상에 대한 위험평가” 증권학회지 제36권 3호(2007) pp. 425-461']

In [135]:
[refer for refer in refer_parsing_ls \
if tuned_dict_ls[0][0] in refer]

['고봉찬 김진우 “발생액 이상현상에 대한 위험평가” 증권학회지 제36권 3호(2007) pp. 425-461',
 '고봉찬 김진우 “발생액 이상현상에 대한 위험평가” 증권학회지 제36권 제3호(2007)pp. 425-461',
 '고봉찬 김진우 “발생액 이상현상에 대한 위험평가” 증권학회지 제36권 3호(2007) pp. 425-461']